In [1]:
# code for storing subject and trial info
from psychopy import gui, visual, core, data, event, logging, clock, colors, layout
# GUI for saving data # Store info about the experiment session
expName = 'game'
exType = 'wet'
expInfo = {'participant': 'X02','type': exType, 'expName' : expName}
# dlg = gui.DlgFromDict(dictionary=expInfo, sortKeys=False, title=expName)
# if dlg.OK == False:
#     core.quit()  # user pressed cancel

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\shaya\anaconda3\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [2]:
import numpy as np
import pandas as pd
import src.realtime_utils as utils
import torch
import pickle
from pathlib import Path
#INIT
filt_ord = 2
freq_limits = np.asarray([[1,100]]) 
freq_limits_names = ['1_100Hz']
sample_duration = 125
sampling_frequency = 250
electrode_names =  ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
filters = utils.init_filters(freq_limits, sampling_frequency, filt_type = 'bandpass', order=filt_ord)
segments, labels, predictions = [], [], []
total_outlier = 0

In [3]:
# init DL model
subject = expInfo['participant']
net = utils.FeatureExtractor()
path = Path(f'./models/{subject}_metamodel.pth')
pretrained_dict = torch.load(path)
net.load_state_dict(pretrained_dict)
net = net.float()
net.eval()

FeatureExtractor(
  (temporal): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=same, bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (spatial): Sequential(
    (0): Conv2d(8, 16, kernel_size=(8, 1), stride=(1, 1), groups=8, bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (seperable): Sequential(
    (0): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=same, groups=16, bias=False)
    (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), padding=same, bias=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
  )
  (avgpool1): AvgPool2d(kernel_size=[1, 4], stride=[1, 4], padding=0)
  (avgpool2): AvgPool2d(kernel_size=[1, 8], stride=[1, 8], padding=0)
  (dropout): Dropout(p=0.1, inplace=False)
  (view): Sequential(
    (0): Flatten()


In [4]:
from pynput.keyboard import Key, Controller
keyboard_game = Controller()

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#psychopy libraries for running the visual cues
from psychopy import gui, visual, core, data, event, logging, clock, colors, layout
import psychopy.iohub as io
from psychopy.hardware import keyboard
#numpy and pd for data storing and manipulation
import numpy as np
import pandas as pd
from numpy.random import  shuffle
# misc libraries to structure the cues properly and save it with date time and stuff
import time
from datetime import date
from pathlib import Path
import random
import os
import time
import msvcrt
import datetime
# lab streaming layer library to capture the data sent by unicorn EEG headset
from pylsl import StreamInlet, resolve_stream

#change path of folders according to your needs
# Data file name stem = absolute path + name; later add .psyexp, .csv, .log, etc

result_path = Path(f'./Expdata/Subjects/'+exType+'/'+expInfo['participant']+'/'+expName+'/')
result_path.mkdir(exist_ok=True, parents=True)

columns=['Time','FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8','AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3',
                                  'Battery','Counter','Validation']

data_dict = dict((k, []) for k in columns)
current_seg = pd.DataFrame()
total_MI_outliers = 0
all_MI_segments, all_MI_labels, predictions = [], [], []

MI_dict = {'MI_segments' : [], 'predictions': []}

# below code is for initializing the streaming layer which will help us capture data later
finished = False
streams = resolve_stream()
inlet = StreamInlet(streams[0])

# Auto updating trial numbers
trial_list = []
for instance in os.scandir(result_path):
        if instance.path.endswith('.csv'):
            length = len(instance.path)
            trial_list.append(int(instance.path[length-5]))

if len(trial_list) == 0:
    session = '01'
elif len(trial_list) < 9 :
    session = len(trial_list) + 1
    session = '0' + str(session)
else :
    session = str(len(trial_list) + 1)

print(f"Conducting {expName} experiment for subject :", expInfo['participant'])
print('No. of Practice Trials before :', 2)
print("Trial Number :", session)

print('Actual Trial')
print('Total number of trials as of now :', int(session) + 2)
results_fname = expInfo['participant']+'_'+str(date.today())+'_'+expName+'_'+ expInfo['type']+'_'+session+'.csv'
print("Saving file as .. ", results_fname)


Fs = 250 # sampling frequency of Unicorn EEG cap
temp = []
pred =1
pred1 = 1
initial = 0
final = 125
prediction = -1
outliers = []
key = False
while not finished:

    sample, timestamp = inlet.pull_sample()
    
    res = [timestamp] + sample 
    data_dict = utils.update_data(data_dict,res)

    if len(data_dict['FZ']) % 125 == 0:
        df, initial, final = utils.segment_dict(initial, final, sample_duration, data_dict)
        segment_filt, out, filters = utils.pre_processing(df, electrode_names, filters, 
                        sample_duration, freq_limits_names, sampling_frequency)
        current_seg = utils.concatdata(current_seg,segment_filt)   
        outliers.append(out)   
        # do prediction with current segment and update number
        if current_seg.shape[1] == 500:
            if sum(outliers) > 0:
                total_MI_outliers +=1
                print('OUTLIER')
                if key:
                    keyboard_game.release(key) 
            else:
                all_MI_segments.append(current_seg)
                prediction = utils.do_prediction(current_seg, net)
                predictions.append(int(prediction[0]))
                print(f"prediction: {prediction[0]}") 
                if key:
                    keyboard_game.release(key)

                if prediction[0] ==0:
                    key = Key.up
                    keyboard_game.press(key)
                elif prediction[0] ==1:
                    key = Key.down
                    keyboard_game.press(key)

#                 if prediction[0] == 0 and pred == 0 and pred1 == 0:
#                     continue
#                 if prediction[0] == 0:
#                     key = Key.up
#                     keyboard_game.press(key)
#                 elif prediction[0] == 1:
#                     key = Key.up
#                     keyboard_game.press(key)
#             pred = prediction[0]
#             pred1 = prediction[0]
            outliers = outliers[1:]

Conducting game experiment for subject : X02
No. of Practice Trials before : 2
Trial Number : 04
Actual Trial
Total number of trials as of now : 6
Saving file as ..  X02_2022-06-23_game_wet_04.csv
OUTLIER
OUTLIER
OUTLIER
OUTLIER
OUTLIER


C:\Users\shaya\anaconda3\lib\site-packages\torch\nn\modules\conv.py:443: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:744.)
  return F.conv2d(input, weight, bias, self.stride,


prediction: 1
prediction: 0
prediction: 0
prediction: 1
prediction: 1
prediction: 1
OUTLIER
OUTLIER
OUTLIER
OUTLIER
OUTLIER
OUTLIER
OUTLIER
OUTLIER
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 1
prediction: 0
prediction: 0
prediction: 0
OUTLIER
OUTLIER
OUTLIER
OUTLIER
prediction: 0
prediction: 0
prediction: 0
prediction: 1
OUTLIER
OUTLIER
OUTLIER
OUTLIER
prediction: 1
prediction: 1
prediction: 1
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 1
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 1
prediction: 1
prediction: 0
prediction: 0
prediction: 1
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
OUTLIER
OUTLIER
OUTLIER
OUTLIER
OUTLIER
prediction: 1
prediction: 0
prediction: 0
prediction: 0
prediction: 0
prediction: 0
predic

In [ ]:
# making dictionary into a dataframe for saving it as csv
record_data = pd.DataFrame.from_dict(data_dict)

#saving MI segments in pickle file
MI_dict = {'MI_segments' : [], 'predictions': []}
MI_dict['MI_segments'] = all_MI_segments
MI_dict['predictions'] = predictions
#f'{subject}_{str(date.today())}_{expName}_{exp_type}_{session}_MIData.pkl'
result_path = Path(f'./Expdata/Subjects/{exType}/{subject}/{expName}')
exp_type = expInfo['type']
MI_fname = f'{subject}_{str(date.today())}_{expName}_{exp_type}_{session}_MIData.pkl'
print("Saving file MI Data file as .. ", MI_fname)
save_file = open(result_path / MI_fname, "wb")
pickle.dump(MI_dict, save_file)
save_file.close()

#fname = Path('./Expdata/Subjects/'+expInfo['participant']+'/'+ expName + '/'+results_fname)
fname = Path('./Expdata/Subjects/'+exType+'/'+expInfo['participant']+'/'+expName+'/'+results_fname)
record_data.to_csv(fname, index = False)
print('Trial Ended')